# Bootstrap: оценка метрик для 2х сплитов

In [176]:
import numpy as np
from random import randrange, shuffle
import random
import sys

In [132]:
data_1 = open("c1.txt", "r")
data_2 = open("c2.txt", "r")

observ_1 = []
observ_2 = []
for i, line in enumerate(data_1):
    l = [int(x) for x in line.strip().split()]
    observ_1.append(np.asarray(l))
    pass

print "The number of observations(split 1): ", len(observ_1)
for i, line in enumerate(data_2):
    l = [int(x) for x in line.strip().split()]
    observ_2.append(np.asarray(l))
    
print "The number of observations(split 2): ", len(observ_2)

The number of observations(split 1):  1000000
The number of observations(split 2):  1000000


In [133]:
#Формируем выборки значений CTR, для подсчета одного значения метрики беру 1000 наблюдений(сессий).
#В качестве 2-ой метрики считаю среднюю долю кликов на сессию

observ_in_group = 1000.0
cnt_group = len(observ_1) / observ_in_group

CTRs_1 = []
CTRs_2 = []
MeanClicks_1 = []
MeanClicks_2 = []

cnt_observ = 0
cur_CTR_1 = 0
cur_CTR_2 = 0
cur_MeanClick_1 = 0
cur_MeanClick_2 = 0

for idx in range(len(observ_1)):
    if cnt_observ == observ_in_group:
        CTRs_1.append(cur_CTR_1 / observ_in_group)
        CTRs_2.append(cur_CTR_2 / observ_in_group)
        MeanClicks_1.append(cur_MeanClick_1 / observ_in_group)
        MeanClicks_2.append(cur_MeanClick_2 / observ_in_group)
        cur_CTR_1 = 0
        cur_CTR_2 = 0
        cur_MeanClick_1 = 0
        cur_MeanClick_2 = 0
        cnt_observ = 0
    
    cur_CTR_1 += 1 if (observ_1[idx].sum() > 0) else 0
    cur_CTR_2 += 1 if (observ_2[idx].sum() > 0) else 0
    cur_MeanClick_1 += observ_1[idx].sum() / float(len(observ_1[idx]))
    cur_MeanClick_2 += observ_2[idx].sum() / float(len(observ_2[idx]))
    cnt_observ += 1    
    
if cnt_observ == observ_in_group:
    CTRs_1.append(cur_CTR_1 / observ_in_group)
    CTRs_2.append(cur_CTR_2 / observ_in_group)
    MeanClicks_1.append(cur_MeanClick_1 / observ_in_group)
    MeanClicks_2.append(cur_MeanClick_2 / observ_in_group)
    
CTRs_1 = np.asarray(CTRs_1)
CTRs_2 = np.asarray(CTRs_2)
MeanClicks_1 = np.asarray(MeanClicks_1)
MeanClicks_2 = np.asarray(MeanClicks_2)

In [134]:
print CTRs_1.mean()
print CTRs_2.mean()
print len(CTRs_1)
print len(CTRs_2)

print MeanClicks_1.mean()
print MeanClicks_2.mean()
print len(MeanClicks_1)
print len(MeanClicks_2)

0.699946
0.670108
1000
1000
0.1619103
0.0862814
1000
1000


In [135]:
print MeanClicks_1[:10]
print MeanClicks_2[:10]

[ 0.1639  0.1585  0.1574  0.1637  0.1554  0.1604  0.1697  0.1638  0.171
  0.1685]
[ 0.0895  0.0854  0.0837  0.0859  0.0858  0.0889  0.0845  0.087   0.0854
  0.0781]


In [166]:
#Процедура Bootstrap, сэмплирование с возвращением
def GetBootstrap(sessions):
    shuffle_idx = [random.randint(0, len(sessions) - 1) for i in range(len(sessions))]
    return sessions[shuffle_idx]

In [179]:
#alpha = 0.05
#возвращает номер принятой гипотезы
def t_criterion(split_1, split_2, t_importance = 1.96):
    t_value = abs(split_1.mean() - split_2.mean()) / \
        np.sqrt((split_1.var(ddof = 1) / float(split_1.shape[0])) + (split_2.var(ddof = 1) / float(split_2.shape[0])))
                  
    return (1 if (t_value > t_importance) else 0)


#вычисляем уровень значимости с помощью bootstrap
def calculate_ASL(split_1, split_2, bootstrap_iter = 10):
    res_criterion = 0
    for it in range(bootstrap_iter):
        if it % 500 == 0:
            sys.stderr.write('\rBootstrap iteration: ' + str(it))
        bs_split_1 = GetBootstrap(split_1)
        bs_split_2 = GetBootstrap(split_2)
        res_criterion += t_criterion(bs_split_1, bs_split_2)
    
#     print res_criterion
    return res_criterion / float(bootstrap_iter)

Считаем ASL для двух метрик:  

In [181]:
print "ASL for CTR: ", calculate_ASL(CTRs_1, CTRs_2, bootstrap_iter=1000000)

 ASL for CTR: 

Bootstrap iteration: 999500

 1.0


In [182]:
print "ASL for MeanCountOfClicks:", calculate_ASL(MeanClicks_1, MeanClicks_2, bootstrap_iter=1000000)

ASL for MeanCountOfClicks:

Bootstrap iteration: 999500

 1.0


Получили, что в обоих случаях мы имеем основание отвергнуть гипотезу о том, что разница в сплитах обусловлена шумами.